#  Implementação


## Simulação

A priori, essa simulação é metade parte em um computador normal (computar os novos valores $\Delta\delta$ e $\Delta\theta$ e parte no computador quântico (obter as saídas a partir das entradas).

Vamos tentar reproduzir os resultados do artigo.

* Os pesos $\theta_{j,k}$ iniciais foram distribuídos aleatoriamente entre $\left[0,2\pi\right)$.
* A taxa de reversão $\delta_{k}$ inicial foi distribuído aleatóriamente entre $\left[-1,1\right]$.
* O objetivo é aprender o basico da operação XOR.
  * Quatro dipo de padrões $\left(x_{1},x_{2}:y^{t}\right)=\left\{\left(0,0:0\right),\left(0,1:1\right),\left(1,0:1\right),\left(1,1:0\right)\right\} $ foi dado a rede a cada aprenzidado.
  * O resultado é uma média de 500 aprendizados.

Para implementar, primeiro precisamos saber como implementar as portas no nosso circuito quântico real. 
Começando com cNOT. Neste caso a variável de controle é outro bit, isto é, a porta do não controlado inverte o qubit alvo, quando o qubit de controle está no estado $\left|1\right\rangle$.

Por tanto precisamos de um qubit de controle para cada um dos nossos qubits de saída.

Como já avaliamos em estudos anteriores,temos:

\begin{equation}
C_{x}=\left(\begin{array}{cccc}
1 & 0 & 0 & 0\\
0 & 1 & 0 & 0\\
0 & 0 & 0 & 1\\
0 & 0 & 1 & 0
\end{array}\right)
\end{equation}

Agora sobre a rotação, temos a rotação padrão em torno do eixo y que é dada por:

\begin{equation}
R_{y}\left(\theta\right)=\left(\begin{array}{cc}
\cos\left(\frac{\theta}{2}\right) & -\sin\left(\frac{\theta}{2}\right)\\
\sin\left(\frac{\theta}{2}\right) & \cos\left(\frac{\theta}{2}\right)
\end{array}\right)
\end{equation}

Aplicando ao nosso qubit temos:

\begin{equation}
R_{y}\left(\alpha\right)f\left(\theta_{j}\right)=\left(\begin{array}{cc}
\cos\left(\frac{\alpha}{2}\right) & -\sin\left(\frac{\alpha}{2}\right)\\
\sin\left(\frac{\alpha}{2}\right) & \cos\left(\frac{\alpha}{2}\right)
\end{array}\right)\left(\begin{array}{c}
\cos\left(\theta_{j}\right)\\
\sin\left(\theta_{j}\right)
\end{array}\right)
\end{equation}

\begin{equation}
R_{y}\left(\alpha\right)f\left(\theta_{j}\right)=\left(\begin{array}{c}
\cos\left(\frac{\alpha}{2}\right)\cos\left(\theta_{j}\right)-\sin\left(\frac{\alpha}{2}\right)\sin\left(\theta_{j}\right)\\
\sin\left(\frac{\alpha}{2}\right)\cos\left(\theta_{j}\right)+\cos\left(\frac{\alpha}{2}\right)\sin\left(\theta_{j}\right)
\end{array}\right)
\end{equation}

Que nos leva ao sistema:

\begin{equation}
\cos\left(\frac{\alpha}{2}\right)\cos\left(\theta_{j}\right)-\sin\left(\frac{\alpha}{2}\right)\sin\left(\theta_{j}\right)=\cos\left(\theta_{j,i}+\theta_{j}\right)
\end{equation}

\begin{equation}
\cos\left(\frac{\alpha}{2}\right)\sin\left(\theta_{j}\right)+\sin\left(\frac{\alpha}{2}\right)\cos\left(\theta_{j}\right)=\sin\left(\theta_{j,i}+\theta_{j}\right)
\end{equation}

Então se usamos a seguinte identidade trigonométrica:

\begin{equation}
\cos\left(a+b\right)=\cos a\cdot\cos b-\sin a\cdot\sin b
\end{equation}

Temos que:

\begin{equation}
\cos\left(\theta_{j,i}+\theta_{j}\right)=\cos\left(\theta_{j,i}\right)\cos\left(\theta_{j}\right)-\sin\left(\theta_{j,i}\right)\sin\left(\theta_{j}\right)
\end{equation}

Logo $\alpha=2\theta_{j,i}$.

Da mesma forma, se usarmos:

\begin{equation}
\sin\left(\theta_{j,i}+\theta_{j}\right)=\cos\left(\theta_{j,i}\right)\sin\left(\theta_{j}\right)+\sin\left(\theta_{j,i}\right)\cos\left(\theta_{j}\right)
\end{equation}

E obtemos o mesmo resultado.

Então vamos usar:
* 2 qubits de entrada
* 1 qubit de saída
* 1 qubit para armezar a variável de controle do qubit de saída
  * Para $n$ saídas, temos $n$ variáveis de controle para o cNOT
* Uma matriz $1 \times 2$ para armazenar os valores de rotação entre a saída e as duas entradas.
  * Para $n$ saídas e $m$ entradas, precisamos de de $n \times m$ valores de rotação.

Como um dos qubits de saída é um dos próprio qubit de entrada, vamos usar 3 qubits. Em geral precisamos dos qubits de entrada, dos qubits com a variável de controle do qubit de saída, e um qubit de saída.

In [1]:
from qiskit import *                             # Importamos as principais funções do Qiskit
from qiskit.visualization import plot_histogram  # A função para plotar o histograma
from qiskit.tools.monitor import job_monitor     # A função pra monitorar o trabalho
import numpy as np                               # Biblioteca de funções científicas

In [2]:
def ge (delta):
    # delta - Taxa de reversão
    return (1/(1+np.power(np.e,-delta)))

In [6]:
# Vamos sortear nossos pesos de rotação e taxa de conversão:

a = np.random.rand(1,3)       # Sorteamos 2 números aleatórios entre 0 e 1
fases = (a[0][0]*np.pi,a[0][1]*np.pi)          # Pra fase convertemos entre 0 e 2.pi
delta = a[0][2] -1            # Pra taxa de reversão convertemos entre -1 e 1
g = ge(delta)                 # calculamos o valor da nossa variável de controle

Para os nossos três qubits no momento temos o mesmo estado:

\begin{equation}
q\left[n\right]=1=e^{i0}
\end{equation}

Queremos que nosso qubit 2 seja:

\begin{equation}
q\left[2\right]=\sqrt{1-g\left(\delta_{1}\right)^{2}}\left|0\right\rangle +g\left(\delta_{1}\right)\left|1\right\rangle 
\end{equation}

Pois queremos que a probabilidae total seja igual a 1, ou seja:

\begin{equation}
\left(\sqrt{1-g\left(\delta_{1}\right)^{2}}\right)^{2}+g\left(\delta_{1}\right)^{2}=1
\end{equation}

Mas se concentrando apenas em $\left|1\right\rangle$, temos que:

\begin{equation}
g\left(\delta_{1}\right)=\sin\left(\theta\right)
\end{equation}

\begin{equation}
\theta=\arcsin\left(g\left(\delta_{1}\right)\right)
\end{equation}

Então podemos realizaar uma rotação:

\begin{equation}
q\left[2\right]'=e^{i\left(0+\theta\right)}=e^{i\cdot0}e^{i\theta}=R_{y}\left(2\theta\right)q\left[2\right]
\end{equation}

In [7]:
del_t=np.arcsin(g)

In [8]:
backend = BasicAer.get_backend('qasm_simulator')     # Escolhomos o backend

# CONSTRUÇÃO do circuito
qc = QuantumCircuit(3,1)                             # Declaramos nossos 3 qubits e o bit clássico que vamos ler o resultado
# 0 - Entrada/saída
# 1 - Entrada
# 2 - Taxa de reversão

qc.ry(del_t*2,2)                                     # Atribuimos nossa taxa de reversão ao qubit 2

Então com nosso sistema preparado, começamos a roda e rede neural. Primeiro vamos passar o valor do qubit preciamos calcular :

\begin{equation}
v_{k}^{O}=S_{k-1}^{O}e^{\theta_{k-1,k}}+\sum_{j=1}^{n}e^{i\theta_{j,k}}S_{j}^{I}
\end{equation}

In [10]:
qc.measure([2], [0])                           # Vamos medir

# EXECUÇÃO
result = execute(qc, backend, shots=1000000).result() # Pegamos o resultado de uma simulação rodando 1000 vezes
counts  = result.get_counts(qc)                    # Contamos os resultados
print(counts)                  

{'1': 206189, '0': 793811}
